In this practical session, we explore the use of the Laplacian operator on a surface, in order to solve heat diffusision, to perform shape matching, or even to compute geodesic distances !

In a first step, we learn how to read and store triangulated surfaces (*meshes*), and how to build the discrete Laplace-Beltrami Operator. 

We then seek to use this operator in different practical scenarios.

# Setup envionment 

Your Python environment should include `numpy`, `scipy`, `matplotlib` and `meshplot`. The latter can be installed from conda using:
><code> conda install -c conda-forge meshplot
</code>

Please download [this zip file](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/data.zip) and place all elements in the same folder as the notebook.

In [ ]:
import numpy as np
import scipy.sparse as sparse


# If you did not put the elements of the zip file in the same folder as the the notebook add the next two lines
# import sys
# sys.path.append("PATH TO THE DIRECTORY WHERE YOU UNZIPED YOUR FILES")
import plot_utils as plu # Follow the above procedure

# 0. Reading and storing a mesh 

Unlike images, there are several ways to represent a discrete (triangulated) surface. In this session, we will stick to the most basic data structure which is made of:
1. A list of vertex coordinates in $\mathbb{R}^3$
2. A list of triangles, each defined by 3 **ordered** vertex indices $i,j,k\in\mathbb{N}$. The order of the vertices define the direction of the normal.

The `OFF` format, or **O**bject **F**ile **F**ormat, stores this information in a `.off` file, which we will use today. 

Open the `bunny.off` file using a text editor, where the structure will look like :
<blockquote>
    <code>OFF
34834 69451 0
-0.0378297 0.12794 0.00447467
-0.0447794 0.128887 0.00190497
...
-0.0400442 0.15362 -0.00816685
3 20463 20462 19669 
3 8845 8935 14299 
...</code>
</blockquote>

As you may already understand, our `.off` file is presented as follows:
<ol>
    <li> First Line: <code>OFF</code> letters</li>
    <li> Second Line:  <code>n m 0</code>, with $n$ the numbers of <b>vertices</b> and $m$ the number of <b>triangles</b></li>
    <li> Next $n$ lines: <code>X Y Z</code> coordinates for each <b>vertex</b></li>
    <li> Next $m$ lines: <code>3 i j k</code> indices of the vertices constituting each <b>face</b></li>
</ol>

In practice, the `OFF` file format can include some more information as you can see on [here](https://en.wikipedia.org/wiki/OFF_(file_format)) if you are interested.

### QUESTION 1
<b> Write the function `read_off` which can read a basic `.off` file.</b>

**Tips** :
<ol>
    <li> You can use the <code>with</code> statement to efficiently process a file: <br>
        <code>with open(filepath, 'r') as f:
   # compute operation on file</code>
    </li>
    <li> <code>f.readline()</code> returns the next line</li>
    <li> <code>text.strip().split()</code> removes breakline characters from <code>text</code> and splits with respect to spaces. Try it with <code>text="1 2 3\n"</code> to see what happens.</li>
</ol>

In [ ]:
def read_off(filepath):
    """
    Reads a simple .off file
    
    Input
    --------------
    filepath : str - path to the .off file
    
    Output
    --------------
    vertices : (n,3) array of vertex coordinates (float)
    faces    : (m,3) array of faces defined by vertices index (integers)
    """
    ## TODO
    # READ THE OFF FILE with path "filepath" and return vertex and faces information
    if len(filepath.split(".")) == 1:
        filepath += ".off"
    with open(filepath, 'r') as f:
        all_lines = f.readlines()
    
    
    n_vertices = int(all_lines[1].split(" ")[0])
    n_faces = int(all_lines[1].split(" ")[1])

    vertices_list = []
    for i in range(n_vertices):
        vertices_list.append([float(x) for x in all_lines[2+i].split(" ")[:3]])

    faces_list = []
    for i in range(n_faces):
        # Be careful to convert to int. Otherwise, you can use np.array(faces_list).astype(np.int32)
        faces_list.append([int(x) for x in all_lines[2+i+n_vertices].split(" ")[1:4]])
    faces = np.array(faces_list)
    vertices = np.array(vertices_list)
    return vertices, faces

As we will have to deal with several components regarding a single mesh, it might be good to create a specific class in which all information will be stored.

In [ ]:
class MyMesh:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)

This way, if we define an instance of `MyMesh` as `mesh = MyMesh(filepath)`, the `__init__` function is automatically run with parameter `filepath` (ignore the `self` parameter).

Then the mesh vertex coordinates can be accessed using `mesh.vertices`and its faces information with `mesh.faces`. These are the *properties* of our class.

Let us load and visualize our bunny:

In [ ]:
mesh = MyMesh('bunny.off')

In [ ]:
plu.plot(mesh)

# 1. Discrete Operators

As seen in class, the Laplacian of a mesh is ubiquitous in geometry processing. The discrete laplacian is defined as
$$
L = A^{-1}W
$$
with $A$ the (diagonal) area matrix, and $W$ the cotangent weight matrix

**Notes for Sparse Matrices**<br>
As the Laplace-Operator is a differential operator, it only locally acts at a point with respect to a local neighborhood. When storing it in a $n\times n$ matrix, this results in numerous zero entries, which are useless to keep in memory.

For both speed and memory efficiency, we use sparse matrices to represent such operators, where only non-zero entries are stored. The most basic structure for a sparse matrix is the **COO**rdinate format, which consists in three lists $I,J,V$ which respectively store the line index, column index and value for each non-zero entry.

For algebra manipulations (matrix vector multiplications, linear system, ...), other format such as **C**ompressed **S**parse **R**ow (CSR) provide a substential speedup compared to COO. You can read about the CSR and CSC format [here](https://en.wikipedia.org/wiki/Sparse_matrix).

In the [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html) library, you can generate COO, CSR or CSC matrices from the $I,J,V$ format using for instance<br>
> <code>scipy.sparse.csr_matrix((V, (I,J)), shape=(N,N))</code>

Note that depending on your python version, you might now have access to the more recent `scipy.sparse.csr_array` functions.

**Notes for numpy linear algebra**<br>
A few tips for simpler reading of numpy
<ol>
    <li> <code>np.matmul(A, B)</code> and <code>A @ B</code> are the same </li>
    <li><code>np.matmum(A, v)</code> and <code>A @ v</code> are the same</li>
    <li><code>np.dot(u, v)</code> and <code>u @ v</code> are the same</li>
    <li> <code>np.transpose(A)</code> and <code>A.T</code> are the same</li>
</ol>

This means <code>A @ B.T @ u</code> is the same as <code>np.matmul(A, np.matmul(np.transpose(B), u))</code>

## Question 2 
<b> Fill the functions <code>compute_faces_areas</code> and <code>area_matrix</code> to respectively compute the area of each face and the vertex-wise area matrix $A$</b>

Recall the area matrix $A$ is a $n\times n$ diagonal matrix where each entry provide the corresponding vertex area defined as:
$$
A_{ii} = \frac{1}{3}\sum_{f\in\mathcal{F}\\ i\in f} \text{Area}(f)
$$

**Tip**: There are multiple ways to compute the area of a triangle $ABC$, one of them being $\frac{1}{2}\|\vec{AB}\times \vec{AC}\|$

In [ ]:
def compute_faces_areas(vertices, faces):
    """
    Compute the area of each face
    
    Input
    --------------
    vertices : (n,3) - vertex coordinates
    faces    : (m,3) - faces defined by vertex indices
    
    Output
    --------------
    faces_areas : (m,) - area of each face
    """
    ## TODO
    # Compute the area of each triangle of the mesh.
    verts_faces = vertices[faces]
    v_1 = verts_faces[:, 0] - verts_faces[:, 1]
    v_2 = verts_faces[:, 0] - verts_faces[:, 2]
    n = np.cross(v_1, v_2)
    faces_areas = 0.5*np.linalg.norm(n, axis=-1)
    return faces_areas

def area_matrix(vertices, faces):
    """
    Compute the diagonal area matrix
    
    Input
    --------------
    vertices : (n,3) - vertex coordinates
    faces    : (m,3) - faces defined by vertex indices
    
    Output
    --------------
    A : (n,n) sparse matrix in DIAgonal format
    """
    #TODO
    # Compute the area of each vertex in the mesh.
    # Use the formula above.
    
    faces_areas = compute_faces_areas(vertices, faces)
    vertex_areas = np.zeros(vertices.shape[0])
    for idx, f in enumerate(faces):
        vertex_areas[f[0]] += faces_areas[idx]/3
        vertex_areas[f[1]] += faces_areas[idx]/3
        vertex_areas[f[2]] += faces_areas[idx]/3
    
    
    # Create a SPARSE diagonal matix from vertex areas
    N = vertices.shape[0]
    A = sparse.dia_matrix((vertex_areas, 0), shape=(N, N))
    return A


<b>Sanity Check : Verify that your total area remains the same whether you sum all face areas or the entries of the area matrix</b>

In [ ]:
faces_areas = compute_faces_areas(mesh.vertices, mesh.faces)

A = area_matrix(mesh.vertices, mesh.faces)
print(A.diagonal().sum() == faces_areas.sum())

## Question 3

Recall the stiffness matrix $W$ is a $n\times n$ sparse matrix. There are multiple ways to compute each entry, one of them being:
$$
W_{ij} = \begin{cases}
\sum_{k\neq i} W_{ik} \quad&\text{if}\quad i=j\\
-\frac{1}{2} \left(\cot(\alpha_{ij}) + \cot(\beta_{ij})\right)\quad&\text{if}\quad ij\in\mathcal{E}\\
0 \quad&\text{else}
\end{cases}
$$

where $\alpha_{ij}$ and $\beta_{ij}$ are the angles opposite to the edge $ij$. Note that actually $\beta_{ij}=\alpha_{ji}$. With other notations you can visualize this face on the image below

![title](https://www.researchgate.net/publication/361577236/figure/fig3/AS:1182638081617922@1658974299594/An-illustration-for-the-cotangent-weights.png)

### Question 3.1 

<b> Fill the function <code>get_cotan_weights</code> to compute the cotangent weights for each face</b>

In [ ]:
def cotan_weights(vertices, faces):
    """
    Compute cotangent weights for each face. Each vertex will carry the cotan of its angle.
    
    For a face [i, j, k], the output will be [alpha_{jk}, alpha_{ki}, alpha_{ij}].
    
    Input
    ----------
    vertices : (n,3) - The vertices of the mesh
    faces : (m,3) The faces of the mesh
    
    Output
    -------
    cotan_weights : (m,3) The cotangent weights for each face
    """
    ### TODO - Compute the cotangent weights
    verts_faces = vertices[faces]
    e_01 = verts_faces[:, 0] - verts_faces[:, 1]
    norm_01 = np.sqrt((e_01* e_01).sum(axis=-1))
    e_02 = verts_faces[:, 0] - verts_faces[:, 2]
    norm_02 = np.sqrt((e_02 * e_02).sum(axis=-1))
    e_12 = verts_faces[:, 1] - verts_faces[:, 2]
    norm_12 = np.sqrt((e_12 * e_12).sum(axis=-1))
    alpha_01 = np.arccos(np.abs((e_02 * e_12).sum(axis=-1))/(norm_02*norm_12))
    alpha_02 = np.arccos(np.abs((e_01 * e_12).sum(axis=-1))/(norm_01*norm_12))
    alpha_12 = np.arccos(np.abs((e_01 * e_02).sum(axis=-1))/(norm_01*norm_02))
    # This can be done in parallel using numpy, or by looping over the faces
    return np.concatenate((alpha_12[:, None], alpha_02[:, None], alpha_01[:, None]), axis=-1)

### Question 3.2 

<b> Fill the function <code>cotan_matrix</code> to compute the cotangent weight matrix $W$</b>

In [ ]:
def cotan(x):
    ## Most of the time it's okay to use 1/np.tan, but remember that it doesn't work for x = pi/2
    return np.cos(x)/np.sin(x)

def cotan_matrix2(vertices, faces):
    ## Very naive version : might crash on computer with less than 16GB of memory (for the bunny), and super slow
    alphas = cotan_weights(vertices, faces)
    n_v = vertices.shape[0]
    W = np.zeros((n_v, n_v))
    for idx_face, (i, j, k) in enumerate(faces):
        for idx_tup, [idx_1, idx_2] in enumerate([(j, k), (k, i), (i, j)]):
            W[idx_1, idx_2] += -0.5*cotan(alphas[idx_face][idx_tup])
            W[idx_2, idx_1] += -0.5*cotan(alphas[idx_face][idx_tup])
    for i in range(n_v):
        W[i, i] -= W[i, :].sum()
    return W

def cotan_matrix_naive(vertices, faces):
    alphas = cotan_weights(vertices, faces)
    n_v = vertices.shape[0]
    # In comment: naive version of building I,J,V, loop over faces, slow because 
    # calculation inside a python loop
    I, J, V = [], [], []
    for idx_face, (i, j, k) in enumerate(faces):
        for idx_tup, [idx_1, idx_2] in enumerate([(j, k), (k, i), (i, j)]):
            value = -0.5*cotan(alphas[idx_face][idx_tup])
            I.append(idx_1)
            J.append(idx_2)
            V.append(value)

            I.append(idx_2)
            J.append(idx_1)
            V.append(value)

            I.append(idx_1)
            J.append(idx_1)
            V.append(-value)

            I.append(idx_2)
            J.append(idx_2)
            V.append(-value)
    
    I_np = np.array(I)
    J_np = np.array(J)
    V_np = np.array(V)
    W = sparse.csc_matrix((V_np, (I_np, J_np)), shape=(n_v, n_v))
    return W

def cotan_matrix(vertices, faces):
    """
    Compute the stiffness matrix
    
    Input
    --------------
    vertices : (n,3) - vertex coordinates
    faces    : (m,3) - faces defined by vertex indices
    
    Output
    --------------
    W : (n,n) sparse matrix in CSC format
    """
    # TODO
    # Compute the entries I,J,V of the stiffness matrix
    # Note that the same pair (i,j) of indices can appear multiple times in I,J
    # The corresponding values in V are then summed by scipy.
    alphas = cotan_weights(vertices, faces)
    n_v = vertices.shape[0]
    # Here, use advantage of numpy arrays
    list_I = []
    list_J = []
    list_V = []
    for idx_tup, (idx_1, idx_2) in enumerate([(1, 2), (2, 0), (0, 1)]):
        value = -0.5*cotan(alphas[:, idx_tup])
        list_I += [faces[:, idx_1], faces[:, idx_2], faces[:, idx_1], faces[:, idx_2]]
        list_J += [faces[:, idx_2], faces[:, idx_1], faces[:, idx_1], faces[:, idx_2]]
        list_V += [value, value, -value, -value]
    I = np.concatenate(list_I, axis=0)
    J = np.concatenate(list_J, axis=0)
    V = np.concatenate(list_V, axis=0)
    W = sparse.csc_matrix((V, (I, J)), shape=(n_v, n_v))
    return W


**Sanity Check : Ensure all terms sum to 0**

In [ ]:
L = cotan_matrix(mesh.vertices, mesh.faces)
np.isclose(np.sum(L), 0)

In [ ]:
## Checking execution times 
import time 
lot_of_memory = False # Only put true if your computer has memory > 16GB.
t1 = time.time()
if lot_of_memory:
    cotan_matrix2(mesh.vertices, mesh.faces)
t2 = time.time()
cotan_matrix_naive(mesh.vertices, mesh.faces)
t3 = time.time()
cotan_matrix(mesh.vertices, mesh.faces)
t4 = time.time()
if lot_of_memory:
    print("Time for Super Naïve : {0:02f}s, Naïve : {1:02f}, All numpy : {2:02f}".format(t2-t1, t3-t2, t4-t3))
else:
    print("Time for Naïve : {1:02f}, All numpy : {2:02f}".format(t2-t1, t3-t2, t4-t3))

Let us now integrate this Laplacian into our mesh class (nothing to change here):

In [ ]:
class MyMesh:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)
        
    def compute_laplacian(self):
        self.A = area_matrix(self.vertices, self.faces)
        self.W = cotan_matrix(self.vertices, self.faces)

The function `compute_laplacian` is called a *method* of the class `MyMesh`, and can be called as follows:

In [ ]:
mesh = MyMesh('bunny.off')
mesh.compute_laplacian()

The `self` parameter actually defined the instance of the class `MyMesh` on which the method is called.
Equivalentely, one could run `MyMesh.compute_laplacian(mesh)` instead of `mesh.compute_laplacian()`.

# 2. Diffusion on Surface 

## 2.1 Exact Diffusion 

Perhaps, the most common use of the Laplace-Beltrami operator is to simulate heat diffusion on an arbitrary surface.

Recall that the Heat Equation is defined as
$$
\frac{\partial u}{\partial t} = \Delta u
$$

Given an initial function $u_0$, the values $u_t$ at time $t$ can be obtained by discretizing the equation using single backward Euler Step, which leads to 
$$
\left(I_d - t\Delta \right) u_t = u_0
$$

Note however that the mathematical laplacian $\Delta$ is <b>negative semidefinite</b>, while our laplacian $L$ is <b>positive semidefinite</b>, and $L\simeq -\Delta$. 

Replacing $\Delta$ by $-L$ and multiplying the left and right side of the equation by $A$, which removes the inverse $A^{-1}$, we obtain

$$
\left(A + tW \right) u_t = A u_0
$$

Therefore, given $u_0$, the value of $u$ at time $t$ can be obtained by solving this **sparse** linear system (as the matrix $A + tW$ we want to invert is sparse).

While the system is made of $n$ equations,  where $n$ is the number of vertices which can grow very large, the fact it is **sparse** enables the use of significanly faster solver than for dense systems of the same size. Note that solving the system **does not** require to actually invert $A + tW$.



## Question 4
<b> Fill the function <code>diffuse_full</code> to compute diffusion of an initial function $f$ for time $t$</b>

**Tip**: Use [`sparse.linalg.spsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spsolve.html)

In [ ]:
def diffuse_full(f, mesh, t):
    """
    Diffuse a function f on a mesh for time t
    
    Input
    --------------
    f       : (n,) - function values
    mesh    : MyMesh - mesh on which to diffuse
    t       : float - time for which to diffuse
    
    Output
    --------------
    f_diffuse : (n,) values of f after diffusion
    """
    # TODO
    # Solve the Diffusion process using the formula above
    left_matrix = mesh.A + t*mesh.W
    right_term = f
    f_diffuse = sparse.linalg.spsolve(left_matrix, right_term)
    return f_diffuse

## Question 4
<b> See what happens when diffusing a dirac function</b>

**Tip**:
<ol>
    <li> Plotting the dirac function centered on index <code>ind</code> can be done with <code>plu.plot(mesh, points=ind)</code> </li>
    <li> Plotting the diffused function <code>f_diffuse</code> can be done with <code>plu.plot(mesh, f_diffuse, colormap='Reds')</code> </li>

In [ ]:
## TODO
ind = 9
dirac = np.zeros(mesh.vertices.shape[0])
dirac[ind] = 1

#plu.plot(mesh, points=ind)
diff_dirac = diffuse_full(dirac, mesh, 0.1)
plu.plot(mesh, diff_dirac, colormap='Reds')

## 2.2 Approximate Diffusion 

As seen in class, the spectrum of the Laplace-Beltrami operator provides a functional basis which generalizes the standard Fourier Analysis to all surfaces.

As the Laplacian is only positive **semidefinite**, its spectrum is obtained by solving the **generalized** eigenvalue problem $W\phi = \lambda A \phi$.

This spectrum consists in eigenvalues - eigenfunctions pairs $\left(\lambda_j, \phi_j\right)_j$.

In practice, we only compute the first $K$ eigenfunctions where $K$ lies in $[10,100]$. This defines an **orthogonal** basis (with respect to the standard inner product defined by $A$), which already provides good approximations of many functions.

In practice, the spectrum is stored as follows:
<ol>
    <li>The eigenvalues $\left(\lambda_j\right)_j$ are stored in a list and seen as a diagonal matrix $\Lambda$</li>
    <li>The eigenvectors $\left(\phi_j\right)_j$ are stored as columns of matrix $\Phi$</li>
</ol>

and proposes the following properties:
<ul>
    <li><b>Orthogonality</b> of the inner product defined by $A$: $\Phi^\top A \Phi = I_K$. This is the discrete translation of $\langle \phi_i, \phi_j\rangle_{A} = \delta_{ij}$, where $\delta$ is here the Kronecker symbol</li>
    <li><b>Eigendecomposition</b>: $L\Phi = \Phi \Lambda$ (or $W\Phi = A\Phi \Lambda$). This is the discrete translation of $L\phi_i=\lambda_i\phi_i$</li>
    <li>The <b>projection</b> $\alpha\in\mathbb{R}^K$ of a function $f\in\mathbb{R}^n$ on the basis is: $\alpha = \Phi^\top A f$. This is the translation of $\alpha_i=\langle\phi_i, f\rangle_A$</li>
    <li>Coefficients $\alpha\in\mathbb{R}^K$ in the basis define a function $f\in\mathbb{R}^n$ on the shape as: $\Phi\alpha = f$. This is the translation of $f=\sum_{i=0}^K \alpha_i\phi_i$</li>
</ul>

Let's add the eigendecomposition of the Laplacian to the mesh class as a new method.

In [ ]:
class MyMesh:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)
        
    def compute_laplacian(self):
        self.A = area_matrix(self.vertices, self.faces)
        self.W = cotan_matrix(self.vertices, self.faces)
        
    def compute_eigendecomposition(self, K):
        self.eigenvalues, self.eigenvectors = sparse.linalg.eigsh(self.W, M=self.A,
                                                                  k=K, sigma=-0.01)
        # The sigma parameter allows to stabilize the eigendecomposition
        

In [ ]:
mesh = MyMesh("bunny.off")
mesh.compute_laplacian()
mesh.compute_eigendecomposition(150)  ## This can take some time

## Question 6
<ol>
    <li><b> Visualize some eigenfunctios of the Laplacian.</b></li>
    <li><b> Visualize the effect of projecting a dirac function on a basis of different size</b>. Use the above explanations to project</li>
</ol>

**Tip** To plot a general function, it is good to use a simple colormap like `"coolwarm"`: <code>plu.plot(mesh, function, colormap="coolwarm")</code>

In [ ]:
## TODO Plot projected dirac
k_dirac = 150
proj = (dirac[:, None] * (mesh.A @mesh.eigenvectors[:, :k_dirac])).sum(axis=0)
rec = ((mesh.eigenvectors[:, :k_dirac])*proj[None, :]).sum(axis=-1)
plu.plot(mesh, rec, colormap='coolwarm')

Coming back to Diffusion, we wish to solve the Diffusion process using spectral analysis. Recall the discretized diffusion equation was
$$
\left(A + tW \right) u_t = A u_0
$$

If we consider $u_t$ to lie in a basis of size $K$ - that is $u_t =\Phi\alpha_t$, multiplying by $\Phi^\top$ on the left gives
$$
\left(I_K + t\Lambda \right) \alpha_t = \Phi^\top A u_0
$$

Since the leftmost term is diagonal, this means $u_t =\Phi\alpha_t$ with the element-wise value:
$$
(\alpha_t)_j = \frac{\beta_j}{1+\lambda_j}
$$

where $\beta=\Phi^\top A u_0$ is actually the **projection** of $u_0$ in the basis.

Note that <b>this only requires elementary matrix multiplications</b> and not solving a linear system.

**Note**: In practice we can have a better approximation by coming back to the original heat equation $\frac{\partial u_t}{\partial t} = \Delta u_t$.

Using $u_t =\Phi\alpha_t$ and $\Delta\Phi=-\Phi\Lambda$, we can obtain $\frac{\partial \alpha_t}{\partial t} = -\Lambda \alpha_t$, with $\Lambda$ being a diagonal matrix. We then obtain
$$
(\alpha_t)_j = \exp(-\lambda_j t) \beta_j
$$

## Question 7
<b> Fill the function <code>diffuse_spectral</code> which diffuses a function f for a time t</b>


In [ ]:
def diffuse_spectral(f, mesh, t, k):
    """
    Diffuse a function f on a mesh for time t
    
    Input
    --------------
    f       : (n,) - function values
    mesh    : MyMesh - mesh on which to diffuse
    t       : float - time for which to diffuse
    k       : int - size of the basis to use for diffusion
    
    Output
    --------------
    f_diffuse : (n,) values of f after diffusion
    """
    # TODO
    # Solve the Diffusion process using spectral analysis.
    # Use the formula above.
    # Note that you should return the function u_t, NOT alpha_t !
    beta = (f[:, None] * (mesh.A @ mesh.eigenvectors[:, :k])).sum(axis=0)
    alpha = np.exp(-mesh.eigenvalues[:k]*t)*beta
    f_diffuse = ((mesh.eigenvectors[:, :k])*alpha[None, :]).sum(axis=-1)
    return f_diffuse

## Question 8
<b>Compare the results of the diffusion using the exact and spectral approach on :</b>
<ol>
    <li> <b>a dirac function</b>.</li>
    <li> <b> a function $\Phi\alpha$ with $\alpha\sim \otimes^k\mathcal{U}(0,1)$ </b>

Note that similarly to standard Fourier analysis, a dirac function require the entire range of frequencies to be represented in the basis.

In [ ]:
## Dirac function, the result is very different (because of low pass effect)
k_diff = 100
diff_dirac_spectral = diffuse_spectral(dirac, mesh, 0.01, k=k_diff)
plu.plot(mesh, diff_dirac_spectral, colormap='Reds')

In [ ]:
## Now showing a random function on the mesh
alpha = np.random.rand(k_diff)
rec_alpha = ((mesh.eigenvectors[:, :k_diff])*alpha[None, :]).sum(axis=-1)
plu.plot(mesh, rec_alpha, colormap='Reds')

In [ ]:
diff_full_alpha = diffuse_full(rec_alpha, mesh, 0.0001)
plu.plot(mesh, diff_full_alpha, colormap='Reds')

In [ ]:
diff_spec_alpha = diffuse_spectral(rec_alpha, mesh, 0.0001, k_diff)
plu.plot(mesh, diff_spec_alpha, colormap='Reds')

In [ ]:
np.linalg.norm(diff_full_alpha - diff_spec_alpha)/(np.linalg.norm(diff_full_alpha)*np.linalg.norm(diff_spec_alpha))

# 3. The Heat Kernel Signature (Bonus, simple)

The [Heat Kernel Signature](http://www.lix.polytechnique.fr/~maks/papers/hks.pdf) (**HKS**) is a pointwise descriptor which provides local information invariant under isometry. In particular, this allows to compute correspondences between near-isometric surfaces.

Given the spectrum $\left(\lambda_j, \phi_j\right)$ of the Laplacian of a shape, the HKS is defined as
$$
HKS(x,t) = C_t\sum_j e^{-\lambda_j t} \phi_j(x)^2
$$

with $C_t^{-1} = \sum_j  e^{-\lambda_j t}$

## Question 9
<b>Fill the `compute_HKS` function to compute HKS descriptor for some time parameters $t$</b>

In [ ]:
def compute_HKS(mesh, n_times, k):
    # Defines a list of time parameters at which to compute HKS
    abs_ev = sorted(np.abs(mesh.eigenvalues[:k]))
    t_list = np.geomspace(4*np.log(10)/abs_ev[-1], 4*np.log(10)/abs_ev[1], n_times)  # (n_times,)
    X_t = np.exp(-mesh.eigenvalues[None, :k]*t_list[:, None])
    C_t = 1./X_t.sum(axis=-1)
    ## TODO COMPUTE HKS
    HKS = C_t * (X_t[None, :] * (mesh.eigenvectors**2)[:, None, :k]).sum(axis=-1)
    return HKS

HKS can later be used as a means to compute shape correspondence. Given two shapes $\mathcal{X}$ and $\mathcal{Y}$, we define the HKS distance between $x\in\mathcal{X}$ and $y\in\mathcal{Y}$ as
$$
d_{HKS}(x,y) = \int_t |HKS_\mathcal{X}(x,t) -  HKS_\mathcal{Y}(y,t)|d\log t
$$

The logarithmic sampling in `compute_HKS` naturally amounts for the $d\log t$ integration, so two embeddings obtained with our function `compute_HKS` can simply be compared using the standard norm of the difference.

Eventually, correspondences $T_{\mathcal{Y}\mathcal{X}}$ from $\mathcal{Y}$ to $\mathcal{X}$ can be obtained by setting
$$
T_{\mathcal{Y}\mathcal{X}}(y) = \underset{y\in\mathcal{Y}}{\text{argmin}}\ d_{HKS}(x,y)
$$

## Question 10
<b>Use HKS to compute correspondences between two surfaces</b>

**Tips**: 
<ol>
    <li> $T_{\mathcal{Y}\mathcal{X}}$ is stored as a $n_\mathcal{Y}$ dimensional array where the $i$-th entry gives the index of the image of $y_i$ in $\mathcal{X}$ </li>
    <li> Plot correspondences $T_{\mathcal{Y}\mathcal{X}}$ using <code>plu.plot_p2p(mesh1, mesh2, T_YX)</code></li>
    
</ol>

In [ ]:
from scipy.spatial import cKDTree

class KNNSearch(object):
    DTYPE = np.float32
    NJOBS = 4

    def __init__(self, data):
        self.data = np.asarray(data, dtype=self.DTYPE)
        self.kdtree = cKDTree(self.data)

    def query(self, kpts, k, return_dists=False):
        kpts = np.asarray(kpts, dtype=self.DTYPE)
        nndists, nnindices = self.kdtree.query(kpts, k=k, workers=self.NJOBS)
        if return_dists:
            return nnindices, nndists
        else:
            return nnindices

    def query_ball(self, kpt, radius):
        kpt = np.asarray(kpt, dtype=self.DTYPE)
        assert kpt.ndim == 1
        nnindices = self.kdtree.query_ball_point(kpt, radius, n_jobs=self.NJOBS)
        return nnindices

In [ ]:
mesh1 = MyMesh("hand1")
mesh2 = MyMesh("hand2")

mesh1.compute_laplacian()
mesh1.compute_eigendecomposition(100)  ## This can take some time


mesh2.compute_laplacian()
mesh2.compute_eigendecomposition(100)  ## This can take some time

In [ ]:
hks1 = compute_HKS(mesh1, 128, 10)
hks2 = compute_HKS(mesh2, 128, 10)
print(hks1.shape)
querier = KNNSearch(hks1)
nn_indices = querier.query(hks2, 1)
# dists = (hks1[:, None]**2).sum(axis=-1) + (hks2[None, :]**2).sum(axis=-1) - 2*np.inner(hks1, hks2)
# nn_indices = np.argmin(dists, axis=0)
plu.plot_p2p(mesh1, mesh2, nn_indices)

# 4. Heat Method for geodesic (BONUS) 

The [Heat method](https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/paperCACM.pdf) uses the diffusion process of a dirac function to compute geodesic distances across a mesh. One advantage of this method is speed and a formulation independant from the specific triangulation of the mesh as long as differential operators are defined (**results are not though**).

Given a point $x$, we start from a diract function $\delta_x$ centered on $x$. The heat method is solved as follows:

<ol>
    <li> Integrate the heat equation with $u_0=\delta_x$, that is solve $\left(A + tW \right) u_t = A \delta_x$. This diffuses a lot heat to points close to $x$ and the value decreases as we for further away points. </li>
    <li> Evaluate the vector field $X=-\nabla u_t / \|\nabla u_t \|$ which gives the (opposite) normalized direction of diffusion, hopefully going *geodesically* towards $x$ </li>
    <li> Solve the Poisson Equation $\Delta \varphi = \nabla\cdot X$ to obtain the geodesic distance $\varphi$ ensuring the distance to $x$ is $0$ by adding the necessary constant</li>
</ol>

The time of diffusion $t$ is usually set as $h^2$ with $h$ the average edge length of the mesh.

## Question 11 (Bonus)
**Implement the heat method usng the gradient and divergence operator we provide**

In [ ]:
def grad_f(f, vertices, faces, normals):
    """
    Compute the gradient of a function on a mesh

    Parameters
    --------------------------
    f          : (n,) function value on each vertex
    vertices   : (n,3) coordinates of vertices
    faces      : (m,3) indices of vertices for each face
    normals    : (m,3) normals coordinate for each face
    face_area : (m,) - Optional, array of per-face area, for faster computation
    use_sym    : bool - If true, uses the (slower but) symmetric expression
                 of the gradient

    Output
    --------------------------
    gradient : (m,3) gradient of f on the mesh
    """
    v1 = vertices[faces[:,0]]  # (m,3)
    v2 = vertices[faces[:,1]]  # (m,3)
    v3 = vertices[faces[:,2]]  # (m,3)

    f1 = f[faces[:,0]]  # (m,)
    f2 = f[faces[:,1]]  # (m,)
    f3 = f[faces[:,2]]  # (m,)

    # Compute area for each face
    face_areas = 0.5 * np.linalg.norm(np.cross(v2-v1,v3-v1),axis=1)  # (m)

   

    grad1 = np.cross(normals, v3-v2)/(2*face_areas[:,None])  # (m,3)
    grad2 = np.cross(normals, v1-v3)/(2*face_areas[:,None])  # (m,3)
    grad3 = np.cross(normals, v2-v1)/(2*face_areas[:,None])  # (m,3)

    gradient = f1[:,None] * grad1 + f2[:,None] * grad2 + f3[:,None] * grad3

    return gradient


def div_f(f, vertices, faces, normals, vert_areas):
    """
    Compute the divergence of a vector field on a mesh

    Parameters
    --------------------------
    f          : (m,3) vector field on each face
    vertices   : (n,3) coordinates of vertices
    faces      : (m,3) indices of vertices for each face
    normals    : (m,3) normals coordinate for each face
    vert_area : (m,) - array of per-vertex area

    Output
    --------------------------
    divergence : (n,) divergence of f on the mesh
    """
    n_vertices = vertices.shape[0]

    v1 = vertices[faces[:,0]]  # (m,3)
    v2 = vertices[faces[:,1]]  # (m,3)
    v3 = vertices[faces[:,2]]  # (m,3)

    grad1 = np.einsum('ij,ij->i', np.cross(normals, v3 - v2) / 2, f)  # (m,)
    grad2 = np.einsum('ij,ij->i', np.cross(normals, v1 - v3) / 2, f)  # (m,)
    grad3 = np.einsum('ij,ij->i', np.cross(normals, v2 - v1) / 2, f)  # (m,)

    I = np.concatenate([faces[:, 0], faces[:, 1], faces[:, 2]])  # (3*m)
    J = np.zeros_like(I)
    V = np.concatenate([grad1, grad2, grad3])

    div_val = sparse.coo_matrix((V, (I, J)), shape=(n_vertices, 1)).todense()

    return np.asarray(div_val).flatten() / vert_areas

In [ ]:
def get_mean_edge_length(mesh):
    e1 = mesh.vertices[mesh.faces[:, 1]] - mesh.vertices[mesh.faces[:, 0]]
    e2 = mesh.vertices[mesh.faces[:, 2]] - mesh.vertices[mesh.faces[:, 0]]
    e3 =  mesh.vertices[mesh.faces[:, 2]] - mesh.vertices[mesh.faces[:, 1]]
    l1 = np.linalg.norm(e1, axis=-1)
    l2 = np.linalg.norm(e2, axis=-1)
    l3 = np.linalg.norm(e3, axis=-1)
    return (np.mean(l1) + np.mean(l2) + np.mean(l3))/3

def face_normals(mesh):
    e1 = mesh.vertices[mesh.faces[:, 1]] - mesh.vertices[mesh.faces[:, 0]]
    e2 = mesh.vertices[mesh.faces[:, 2]] - mesh.vertices[mesh.faces[:, 0]]
    cross = np.cross(e1, e2)
    mesh_normals = cross/np.linalg.norm(cross, axis=-1, keepdims=True)
    return mesh_normals

def heat_distance(mesh, idx):
    dirac = np.zeros(mesh.vertices.shape[0])
    dirac[idx] = 1
    # Computing diffusion of dirac 
    h = get_mean_edge_length(mesh)
    diff_h = diffuse_full(dirac, mesh, h**2)
    # Computing face normals
    mesh_normals = face_normals(mesh)

    # Heat method
    grad_diff = grad_f(diff_h, mesh.vertices, mesh.faces, mesh_normals)
    X = -grad_diff/np.linalg.norm(grad_diff, axis=-1, keepdims=True)
    div_X = div_f(X, mesh.vertices, mesh.faces, mesh_normals, mesh.A.diagonal())
    # Linear system (don't forget the mass matrix when discretizing!) 
    left = mesh.W 
    right = mesh.A @ div_X
    sol = sparse.linalg.spsolve(left, right)
    dists = sol - np.min(sol)
    return dists, h

import copy
import matplotlib.pyplot as plt
def line_plot(dists, h):
    # Simple function to plot the level set of geod distances
    plot_func = copy.deepcopy(dists)
    eps = dists.max()/15
    color_plot = plt.cm.coolwarm((plot_func-plot_func.min())/(plot_func.max()-plot_func.min()))
    for i in range(15):
        color_plot[np.logical_and(dists>i*eps-h/2, dists<i*eps+h/2), :] = 0
    return color_plot

In [ ]:
geod_dists, h = heat_distance(mesh, ind)

In [ ]:
# Plotting the resulting distance
plu.plot(mesh, geod_dists, colormap='coolwarm')

In [ ]:
# Showing the level set, we're supposed to see "circles" of equidistant points to the chosen vertex
plot_func = line_plot(geod_dists, h)
plu.plot(mesh, cmap=plot_func[:, :3])